# IBM:DSP Capstone Project

## Territory Market Development

### Introduction / Business Problem

   
A new technical sales representative for a winery products company has been assigned a new territory. She is to contact and develop sales relationships with wineries in the Pacific Northwestern United States. She seeks to establish new relationships and develop her new sales territory. However, the Pacific Northwestern United States includes both Washington State and Oregon State. The area encompasses vast areas and not all areas are viticulture areas or vineyards. How can she best expand her territory to include new clients?

### Data  

We will use an existing winery list as a substitute for an actual existing client list from data world. We can gather this list by using a web scrape, an SQL query, or pandas. Then, we'll leverage the Foursquare API to create a second list of wineries. We'll link the location with geopy and build an overlay. We'll separate regions using k-means clustering.

#### Data World Data Set

The first data set resides at <a href="https://data.world/arthur/wineries">data world</a>. I'll first create an account and attempt to download the file as an excel sheet. This was easy enough to complete. I went into the files and looked around at the data. We are examining the Pacific Northwest Territory so I had to download the file for the <a href="https://query.data.world/s/nib6nc7kfdk7vhbaypnipvdogzcqz2">USA</a>.

Now that we have downloaded the file from the website, let's look at an alternative using Pandas

In [53]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/r4ahdp3vbrclyyim5siydrszdc6rrx')
df = df.drop(['Unnamed: 3'], axis=1) # Drop the unnamed column
df = df.drop(['Web Site'], axis=1) #Drop web site, come back put in one line
df['long'] = "" # add Longitude Column
df['lat'] = "" # add Latitude Column
df.rename(columns = {'Winery Name':'Winery'}, inplace = True) #Rename
df.head()

,Winery,State,long,lat
0,14 Hands,WA,,
1,Abacela Vineyards & Winery,OR,,
2,Abarbanel Wine Co.,NY,,
3,Abbott Winery,CA,,
4,Abeja,WA,,


In [52]:
#To select rows which have WA or OR as the State
states = ['WA','OR']
df = df[df.State.isin(states)]
df.reset_index(drop=True) #Drop the old index

,Winery,State,Web Site,long,lat
0,14 Hands,WA,www.14handswine.com,-119.747,46.2111
1,Abacela Vineyards & Winery,OR,www.abacela.com,,
2,Abeja,WA,www.abeja.net,,
3,Academy Wines,OR,NaN,,
4,Acme Wineworks,OR,NaN,,
...,...,...,...,...,...
414,Yamhill Valley Vineyards,OR,www.yamhill.com,,
415,Yellow Hawk Cellar,WA,NaN,,
416,Youngberg Hill Vineyards,OR,www.youngberghill.com,,
417,Zefina Winery,WA,www.zefina.com,,


In [55]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# Need to come back here and work the loop, concatenate in.
address = '14 Hands Winery'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

Solving environment: ...working... done

# All requested packages already installed.

46.211056 -119.7465957


#### So this portion is working. Now, how do I get this loop iterated and back into the df

In [56]:
for x in range(len(df)):
    try:
        geocode_result = geolocator.geocode(df.Winery)
        df['lat'][x] = geocode_result.latitude
        df['long'][x] = geocode_result.longitude
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occured.", e )
df.head()

Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'latitude'
Unexpected error occured. 'NoneType' object has no attribute 'la

KeyboardInterrupt: 

In [ ]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

address = 'Portland OR, USA'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

In [ ]:
# Create a map using folium
map_PNW = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=6)
map_PNW

Let's see if we can get geopy to integrate with the four-square data.

### Methodology     

   * The main component of the report where I discuss and describe any exploratory data analysis, any inferential statistical testing, and what machine learning was used and why. 


### Results

   * This is where I discuss the results.

### Discussion

   * This is where I discuss any observations I noted and any recommendations I can make based on the results.

### Conclusion 

   * This is where I conclude the report.